# Worksheet 1: File locations and pre-processing

The following exercises demonstrate some of the tools available for data analysis, and how to prepare PRECIS output for analysis. This can be time consuming for large amounts of data, so in this worksheet a small subset is used to demonstrate the steps involved. In the worksheets that follow, data that has already been processed will be used.

PRECIS output data tables are in PP format, a Met Office binary data format. This worksheet converts data to NetCDF format (a standard format in climate science) in order that it can be used in post processing packages such as Python and the python Library [IRIS](http://scitools.org.uk/iris/docs/latest/index.html).


__Note:__ In the boxes where there is code or where you are asked to type code, click in the box press 'ctrl' + 'enter' to run the code.

__Note:__ An explanation '!' mark is needed to run commands on the shell, and is noted where needed.

__Note:__ Anything after the character `#` is just a comment and does not affect the command being run.


## 1.1 Data locations and file names


**Identify and list the names of PRECIS output data in PP format using
standard Linux commands.**

The datasets used within these worksheets are made available through the notebook in order to providie quick and easy access for the purpose of this training. However the controls learnt in this worksheet provide useful context for future work in a linux and unix scripting environment.

The dataset used here is a three year subset of monthly PRECIS data over south east Asia driven by the HadCMQ0 GCM.


* (a) Firstly, find out what location you are currently in by using the ***pwd*** command; _pwd_ stands for *print working directory*.

Type ***!pwd*** on the box below.


In [ ]:
# Type !pwd below and press 'ctrl' + 'enter'

* (b) List the contents of this directory; ***ls*** stands for **list** and using the ***-l*** option gives a longer listing with more information, such as file and size and modification date.

Type below ***!ls*** on one line and then the next ***!ls -l*** and press **ctrl + enter**.


In [ ]:
# Type !ls on one line and then !ls -l and press 'ctrl' + 'enter'.


* (c) Move to the directory (i.e. folder) called *practise_ppfiles/cahpa'*. This directory contains monthly output data from experiment with RUNID *cahpa*; ***cd*** command stands for *change directory*.

Type below ***%cd practise_ppfiles/cahpa*** and press *ctrl + enter*. In notebooks *%* is used to execute in commands in the shell.


In [ ]:
# Type %cd practise_ppfiles/cahpa and press 'ctrl' + 'enter'.

* (d) List the contents of this directory; remeber ***ls*** command stands for *list* and using the ***-l*** option gives a longer listing with more information, such as file and size and modification date.



In [ ]:
# Type shell command here not forgetting the ! and press 'ctrl' + 'enter'.

* (e) List all the files containing data for September.

Type below ***!ls \*sep\****

How many files contain data for September?

__Note:__ The character * translates as *any characters*

In [ ]:
# Type !ls *sep* and press 'ctrl' + 'enter'.

* (f) List all the files containing data from 1982 (i.e. all files which begin **cahpaa.pmi2**)

Type below ***!ls cahpaa.pmi2???.pp***

Note:The character *?* translates as *any single character*



In [ ]:
# Type !ls !ls cahpaa.pmi2???.pp and press 'ctrl' + 'enter'.

* (g) Move up two levels in the directory tree and list the directories.

Type the following commands one at a time and immediately after press *ctrl + enter* to execute the command

***%cd ..***

***%cd ..***

***!pwd***

***!ls***

The directories daily and monthly contain data used in the worksheets which follow this one.

In [ ]:
# Type one command a time and press 'ctrl' + 'enter' each time.

## 1.2 Introduction to python and IRIS

Python is an interpreted, object-oriented, high-level programming language. Python supports modules and packages, which encourages program modularity and code reuse. 


We also use the python library IRIS, which is written in Python and was setup and maintained by the Met Office. "Iris seeks to provide a powerful, easy to use, and community-driven Python library for analysing and visualising meteorological and oceanographic data sets."

For a brief introduction to IRIS and the cube formatting please read this Introduction page here: http://scitools.org.uk/iris/docs/latest/userguide/iris_cubes.html

For further future reference please refer to the IRIS website: http://scitools.org.uk/iris/docs/latest/index.html

* The following code is a simple example of how to load a NetCdf file into an Iris cube and then print its metadata strucuture

In [ ]:
# Ensure you haver iris 1.10 or greater
import iris

# this prints the version of the Iris library you are using
print iris.__version__

# you must adjust this to the path of your sample data
sample_data = '/path/to/sample_data/sample_file'

# this loads your sample data into an Iris cube structure
cube = iris.load_cube(sample_data)

# This prints out the cube metadata structure
print cube

/opt/scitools/environments/default/2017_06_07-1/lib/python2.7/site-packages/iris/__init__.py:480: IrisDeprecation: iris.config.SAMPLE_DATA_DIR was deprecated in v1.10.0 and will be removed in a future Iris release. Install the 'iris_sample_data' package.
  warn_deprecated(wmsg)


ValueError: Sample data file(s) at '/opt/scitools/environments/default/2017_06_07-1/lib/python2.7/site-packages/iris/sample_data/air_temp.pp' not found.
NB. This function is only for locating files in the iris sample data collection. It is not needed or appropriate for general file access.

## 1.3 Remove the rim from PP fields
The edges (or rim) of RCM outputs are biased due to the linear relaxation used on certain variables to apply the GCM lateral boundary conditions. This
rim of 8 grid points from each edge needs to be excluded from any
analysis.

* Remove the 8-point rim from all data in the `practise_ppfiles/cahpa` directory (with the option of automatically deleting the original full-sized files).

First of all in the first code box we will change to the 'practice_ppfiles/cahpa' directory, click in the box and press 'ctrl' + 'enter'.

In [ ]:
# click in the box and press 'ctrl' + 'enter'
%cd practise_ppfiles/cahpa

/data/local/ssadri/PRECIS_WORK/jupyter/practise_ppfiles/cahpa


Then this next code box removes the 8-point ri, from all data in that directory. Click in the box and press 'ctrl' + 'enter'.

In [4]:
import iris
import glob
import os

# find all the files from which to remove the rim
flist = glob.glob('*.rr8.pp')

# checks if output directory exists, if not creates a directory
if not os.path.exists('rr8_removed'):
    os.mkdir('rr8_removed')

for fn in flist:
    # This will load all the variables in the file into a CubeList
    datacubes = iris.load(fn)
    tempcubelist = []
    for cube in datacubes:
        temprimcubelist = []
        # In case the data are more than 2 dimensional, grab a 2d slice
        for yx_slice in cube.slices(['grid_latitude','grid_longitude']):
            norimcube = yx_slice[8 : -8 , 8 : -8]
            temprimcubelist.append(norimcube)
        # Merge the individual 2d slices into a single cube
        trimmedcube = iris.cube.CubeList(temprimcubelist).merge_cube()
        tempcubelist.append(trimmedcube)
    # Write out the trimmed data file
    iris.save(tempcubelist, 'rr8_removed/' + fn, append=True)
    print 'The 8-point rim has been removed for: ' + fn

The 8-point rim has been removed for: cahpaa.pmi2jan.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi2may.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi3aug.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi3jan.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi3mar.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi3sep.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi3jun.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi3may.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi2jun.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi1nov.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi2nov.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi3feb.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi2dec.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi1jul.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi2feb.rr8.pp
The 8-point rim has been removed for: cahpaa.pmi3jul.rr8.pp
The 8-point rim has been removed for: ca

## 1.4 Select variables and convert PP files to NetCDF

The monthly data we are using has multiple variables in each file, we can use a Iris to separate the variables and save them as netCDF files.

In this example you will see a reference to the cubes attibutes 'STASH'. STASH codes are used as a storage handling system for all the variables that the PRECIS model and Met Office UM model provides. Each stash code refers to a variable, therefore in this example below: 03236 - air temperature, 16222 - air pressure, 05216 - precipitation. You will notice that the files have been saved with the relevant STASH code in this example. However you could name the files after the variable names, ie temp, precip, slp for ease.

* Separate the variables in all of the monthly files into separate directories and save as netCDF files.

Click in the box below and press 'ctrl' + 'enter' to run the code.


In [5]:
import iris
import glob
import os

flist = glob.glob('rr8_removed/' + '*.rr8.pp')

for fn in flist:
    # This will load all the variables in the file into a CubeList
    datacubes = iris.load(fn)
    
    for cube in datacubes:
        # get the STASH code
        cubeSTASH = cube.attributes['STASH'] 

        # create a directory based on the STASH code
        dirname = str(cubeSTASH.section).zfill(2)+str(cubeSTASH.item).zfill(3)
        # checks if directory exists, if not creates a directory
        if not os.path.exists(dirname):
            os.mkdir(dirname)
            
        # for saving replace the *.pp file extension with *.nc
        outfile = fn.replace('.pp','.' + dirname + '.nc')
        
        # save the merged data cube
        iris.save(cube, dirname + "/" + outfile[12:])

print 'All netCDF files have been saved to relevant stash code directory'

/opt/scitools/environments/default/2017_06_07-1/lib/python2.7/site-packages/iris/fileformats/netcdf.py:2281: IrisDeprecation: NetCDF default saving behaviour currently assigns the outermost dimensions to unlimited. This behaviour is to be deprecated, in favour of no automatic assignment. To switch to the new behaviour, set iris.FUTURE.netcdf_no_unlimited to True.
  warn_deprecated(msg)


All netCDF files have been saved to relevant stash code directory


* For each variable (Temperature, Precipitation, Surface Pressure) put the monthly files into a single cube and save as netCDF file.

The monthly files are for the years 1981, 1982, and 1983, hence the file name saved to including 1981_1983.

Click in the box below and press 'ctrl' + 'enter' to run the code.


In [ ]:
import iris
import glob

stash_codes = ['03236','05216','16222']

# loop over each directory stash code
for stash in stash_codes:
   
    # load the file names into the variable flist
    flist = glob.glob(stash + '/' + '*.nc')

    cubelist = [] # an empty array to append the monthly cubes to
    for fn in flist:
        # There is only on cube so using single cube load function
        datacube = iris.load_cube(fn)
        cubelist.append(datacube)
    
    # Merge all the cubes in the list created above
    mergedcube = iris.cube.CubeList(cubelist).merge_cube()
    
    # Can save the data direclty in netCDF format
    outfile = stash + '/cahpaa.pm.1981_1983.rr8.' + stash + '.nc'
    

    iris.save(mergedcube, outfile)
    
print 'Each variable has been saved into a single netCDF file in its relevant stash directory.'
print

/opt/scitools/environments/default/2017_06_07-1/lib/python2.7/site-packages/iris/fileformats/cf.py:1143: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


Each variable has been saved into a single netCDF file in its relevant stash directory.

